In [ ]:
## Experiments: Training the entire dense layer

In [1]:
from Executor import *
# Reset the model, and run same finetuning using precomputed conv. model outputs. See if they match...
executor = ExecutorBuilder().\
    with_runID("first").\
    and_().\
    with_Vgg16().\
    and_().\
    train_batch_size(128). \
    and_(). \
    learn_rate(0.001).\
    and_().\
    data_on_path("../data/full/").\
    build()

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


Pretrained Vgg16 model loaded.
Found 6255 images belonging to 3 classes.
Found 1549 images belonging to 3 classes.
initialized training data from: ../data/full/train
initialized validation data from: ../data/full/valid
found number of softmax classes: 3


In [2]:
executor.load_model_from_file() # loads the model from the previous ipynb file
executor.load_precomputed_conv_models()
# Get the linear model 
linearModel = executor.get_rescaled_fc_model(new_dropout=executor.dropout)

# Set all layers being trainable
for layer in linearModel.layers:
    if layer.trainable:
        print("found trainable layer: ",layer)


loading precomputed conv. outputs...
done...
getting rescaled fc model...
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
done...
('updating dropout from: ', 0.5, ' to: ', 0.5)
('found trainable layer: ', <keras.layers.pooling.MaxPooling2D object at 0x7f499c0787d0>)
('found trainable layer: ', <keras.layers.core.Flatten object at 0x7f495c2ef750>)
('found trainable layer: ', <keras.layers.core.Dense object at 0x7f4959ab8ad0>)
('found trainable layer: ', <keras.layers.core.Dropout object at 0x7f4959ab8690>)
('found trainable layer: ', <keras.layers.core.Dense object at 0x7f4974af7d50>)
('found trainable layer: ', <keras.layers.core.Dropout object at 0x7f4959b1de90>)
('found trainable layer: ', <ker

In [3]:
linearModel.compile(optimizer=Adam(lr=0.000001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 5, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 7s - loss: 1.9772 - acc: 0.3952 - val_loss: 1.1805 - val_acc: 0.4913
Epoch 2/5
6255/6255 [==============================] - 7s - loss: 1.7330 - acc: 0.4230 - val_loss: 1.0740 - val_acc: 0.5029
Epoch 3/5
6255/6255 [==============================] - 7s - loss: 1.5726 - acc: 0.4456 - val_loss: 1.0362 - val_acc: 0.5216
Epoch 4/5
6255/6255 [==============================] - 7s - loss: 1.5259 - acc: 0.4571 - val_loss: 0.9929 - val_acc: 0.5339
Epoch 5/5
6255/6255 [==============================] - 7s - loss: 1.3903 - acc: 0.4811 - val_loss: 0.9711 - val_acc: 0.5455


In [4]:
# Now try to finetune the model, then maybe set the all dense layers trainable

In [4]:
linearModel.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 5, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 7s - loss: 1.2040 - acc: 0.5133 - val_loss: 0.8693 - val_acc: 0.5875
Epoch 2/5
6255/6255 [==============================] - 7s - loss: 0.9192 - acc: 0.5859 - val_loss: 0.8246 - val_acc: 0.6133
Epoch 3/5
6255/6255 [==============================] - 7s - loss: 0.7683 - acc: 0.6512 - val_loss: 0.7899 - val_acc: 0.6294
Epoch 4/5
6255/6255 [==============================] - 7s - loss: 0.6616 - acc: 0.7165 - val_loss: 0.7564 - val_acc: 0.6630
Epoch 5/5
6255/6255 [==============================] - 7s - loss: 0.5715 - acc: 0.7604 - val_loss: 0.7274 - val_acc: 0.6766


In [5]:
# BAD IDEA... to increase the learn_rate, if anything, the learn_rate should be decreased.
linearModel.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 5, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 7s - loss: 0.8848 - acc: 0.5906 - val_loss: 0.7973 - val_acc: 0.5997
Epoch 2/5
6255/6255 [==============================] - 7s - loss: 0.6228 - acc: 0.7162 - val_loss: 0.7866 - val_acc: 0.6475
Epoch 3/5
6255/6255 [==============================] - 7s - loss: 0.3915 - acc: 0.8374 - val_loss: 0.7836 - val_acc: 0.6888
Epoch 4/5
6255/6255 [==============================] - 7s - loss: 0.2783 - acc: 0.8878 - val_loss: 0.9418 - val_acc: 0.6940
Epoch 5/5
6255/6255 [==============================] - 7s - loss: 0.2044 - acc: 0.9263 - val_loss: 0.8864 - val_acc: 0.7114
